In [1]:
!pip install skl2onnx onnx onnxmltools

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

from sklearn.linear_model import LogisticRegression
clr = LogisticRegression()
clr.fit(X_train, y_train)

/usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [3]:
clr.predict([[5.1, 3.5, 1.4, 0.2]])

array([0])

In [4]:
%%writefile iris_classifier_onnx.py

import numpy
import bentoml
from bentoml.adapters import DataframeInput
from bentoml.artifact import OnnxModelArtifact

@bentoml.artifacts([OnnxModelArtifact('model')])
@bentoml.env(auto_pip_dependencies=True)
class IrisClassifierOnnx(bentoml.BentoService):
    
    @bentoml.api(input=DataframeInput())
    def predict(self, df):
        input_data = df.to_numpy().astype(numpy.float32)
        input_name = self.artifacts.model.get_inputs()[0].name
        output_name = self.artifacts.model.get_outputs()[0].name
        return self.artifacts.model.run([output_name], {input_name: input_data})[0]  

Overwriting iris_classifier_onnx.py


In [5]:
from iris_classifier_onnx import IrisClassifierOnnx
from skl2onnx.common.data_types import FloatTensorType
import onnxmltools



initial_type = [('float_input', FloatTensorType([None, 4]))]
onnx_model = onnxmltools.convert_sklearn(clr, initial_types=initial_type)

svc = IrisClassifierOnnx()
svc.pack('model', onnx_model)

saved_path = svc.save()

[2020-05-17 11:46:24,560] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.5. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-17 11:46:25,436] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.5. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check
creating BentoML-0.7.5+21.g2b3d2c8.dirty
creating BentoML-0.7.5+21.g2b3d2c8.dirty/BentoML.egg-info
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/artifact
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/bundler
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/cli
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/clipper
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/configuration
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/configuration/__pycache__
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/deployment
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/deployment/aws_lambda
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/deployment/sagemaker
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/handlers
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/marshal
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/migrations
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bento

creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/media-typer
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/merge-descriptors
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/methods
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/mime
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/mime-db
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/mime-types
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/mime/src
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/ms
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/negotiator
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/negotiator/lib
creating BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/on-finished
creating BentoML-0.7.5+21.g2b3d2c8.dirty/

copying bentoml/artifact/tf_savedmodel_artifact.py -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/artifact
copying bentoml/artifact/xgboost_model_artifact.py -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/artifact
copying bentoml/bundler/__init__.py -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/bundler
copying bentoml/bundler/bundler.py -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/bundler
copying bentoml/bundler/config.py -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/bundler
copying bentoml/bundler/loader.py -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/bundler
copying bentoml/bundler/py_module_utils.py -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/bundler
copying bentoml/bundler/templates.py -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/bundler
copying bentoml/bundler/utils.py -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/bundler
copying bentoml/cli/__init__.py -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/cli
copying bentoml/cli/aws_lambda.py -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/cli
copying bentoml/cli/aws_s

...

copying bentoml/yatai/web/dist/node_modules/negotiator/lib/mediaType.js -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/negotiator/lib
copying bentoml/yatai/web/dist/node_modules/on-finished/HISTORY.md -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/on-finished
copying bentoml/yatai/web/dist/node_modules/on-finished/LICENSE -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/on-finished
copying bentoml/yatai/web/dist/node_modules/on-finished/README.md -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/on-finished
copying bentoml/yatai/web/dist/node_modules/on-finished/index.js -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/on-finished
copying bentoml/yatai/web/dist/node_modules/on-finished/package.json -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/on-finished
copying bentoml/yatai/web/dist/node_modules/parseurl/HISTORY.md -> BentoML-0.7.5+21.g2b3d2c8.dir

copying bentoml/yatai/web/dist/node_modules/protobufjs/cli/targets/proto3.js -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/protobufjs/cli/targets
copying bentoml/yatai/web/dist/node_modules/protobufjs/cli/targets/static-module.js -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/protobufjs/cli/targets
copying bentoml/yatai/web/dist/node_modules/protobufjs/cli/targets/static.js -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/protobufjs/cli/targets
copying bentoml/yatai/web/dist/node_modules/protobufjs/cli/wrappers/amd.js -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/protobufjs/cli/wrappers
copying bentoml/yatai/web/dist/node_modules/protobufjs/cli/wrappers/closure.js -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/protobufjs/cli/wrappers
copying bentoml/yatai/web/dist/node_modules/protobufjs/cli/wrappers/commonjs.js -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dis

copying bentoml/yatai/web/dist/node_modules/protobufjs/src/encoder.js -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/protobufjs/src
copying bentoml/yatai/web/dist/node_modules/protobufjs/src/enum.js -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/protobufjs/src
copying bentoml/yatai/web/dist/node_modules/protobufjs/src/field.js -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/protobufjs/src
copying bentoml/yatai/web/dist/node_modules/protobufjs/src/index-light.js -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/protobufjs/src
copying bentoml/yatai/web/dist/node_modules/protobufjs/src/index-minimal.js -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/protobufjs/src
copying bentoml/yatai/web/dist/node_modules/protobufjs/src/index.js -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/protobufjs/src
copying bentoml/yatai/web/dist/node_modules/protobufjs/src/ma

copying bentoml/yatai/web/dist/node_modules/range-parser/LICENSE -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/range-parser
copying bentoml/yatai/web/dist/node_modules/range-parser/README.md -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/range-parser
copying bentoml/yatai/web/dist/node_modules/range-parser/index.js -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/range-parser
copying bentoml/yatai/web/dist/node_modules/range-parser/package.json -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/range-parser
copying bentoml/yatai/web/dist/node_modules/raw-body/HISTORY.md -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/raw-body
copying bentoml/yatai/web/dist/node_modules/raw-body/LICENSE -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/raw-body
copying bentoml/yatai/web/dist/node_modules/raw-body/README.md -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai

copying bentoml/yatai/web/dist/node_modules/statuses/HISTORY.md -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/statuses
copying bentoml/yatai/web/dist/node_modules/statuses/LICENSE -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/statuses
copying bentoml/yatai/web/dist/node_modules/statuses/README.md -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/statuses
copying bentoml/yatai/web/dist/node_modules/statuses/codes.json -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/statuses
copying bentoml/yatai/web/dist/node_modules/statuses/index.js -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/statuses
copying bentoml/yatai/web/dist/node_modules/statuses/package.json -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/statuses
copying bentoml/yatai/web/dist/node_modules/toidentifier/LICENSE -> BentoML-0.7.5+21.g2b3d2c8.dirty/bentoml/yatai/web/dist/node_modules/toid

### Use BentoService with BentoML CLI


Use bentoml get to list all versions of BentoService, including the version tag will display additional information and metadata

In [7]:
!bentoml get IrisClassifierOnnx

BENTO_SERVICE                             AGE                          APIS                       ARTIFACTS
IrisClassifierOnnx:20200517114625_DB7077  8.94 seconds                 predict<DataframeHandler>  model<OnnxModelArtifact>
IrisClassifierOnnx:20200517114420_060306  2 minutes and 13.24 seconds  predict<DataframeHandler>  model<OnnxModelArtifact>
IrisClassifierOnnx:20200516145155_F82EDF  20 hours and 54 minutes      predict<DataframeHandler>  model<OnnxModelArtifact>
IrisClassifierOnnx:20200516144844_4E14BE  20 hours and 57 minutes      predict<DataframeHandler>  model<OnnxModelArtifact>
IrisClassifierOnnx:20200516144520_BCC8AC  21 hours and 1 minute        predict<DataframeHandler>  model<OnnxModelArtifact>
IrisClassifierOnnx:20200514230108_560ED6  2 days and 12 hours          predict<DataframeHandler>  model<OnnxModelArtifact>
IrisClassifierOnnx:20200514225741_2EFAF6  2 days and 12 hours          predict<DataframeHandler>  model<OnnxModelArtifact>
IrisClassifierOnnx:202005142251

In [8]:
!bentoml get IrisClassifierOnnx:latest

[2020-05-17 11:46:53,198] INFO - Getting latest version IrisClassifierOnnx:20200517114625_DB7077
{
  "name": "IrisClassifierOnnx",
  "version": "20200517114625_DB7077",
  "uri": {
    "type": "LOCAL",
    "uri": "/Users/bozhaoyu/bentoml/repository/IrisClassifierOnnx/20200517114625_DB7077"
  },
  "bentoServiceMetadata": {
    "name": "IrisClassifierOnnx",
    "version": "20200517114625_DB7077",
    "createdAt": "2020-05-17T18:46:41.959251Z",
    "env": {
      "condaEnv": "name: bentoml-IrisClassifierOnnx\nchannels:\n- defaults\ndependencies:\n- python=3.7.3\n- pip\n",
      "pipDependencies": "onnxruntime\nbentoml==0.7.5\nnumpy==1.18.4",
      "pythonVersion": "3.7.3"
    },
    "artifacts": [
      {
        "name": "model",
        "artifactType": "OnnxModelArtifact"
      }
    ],
    "apis": [
      {
        "name": "predict",
        "handlerType": "DataframeHandler",
        "docs": "BentoService API",
        "handlerConfig": {
          "orient": "records",
          "typ": "f

In [10]:
!bentoml run IrisClassifierOnnx:20200517114625_DB7077 predict --input '[[5.1, 3.5, 1.4, 0.2]]'

[2020-05-17 11:47:18,195] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.5. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-17 11:47:18,207] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.7.5,  but loading from BentoML version 0.7.5+21.g2b3d2c8.dirty
[2020-05-17 11:47:18,215] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image 

### Model Serving via REST API


Note: Running as local rest api server does not work with Google Colab, please copy this notebook to run it locally

In [11]:
!bentoml serve {saved_path}

[2020-05-17 11:47:53,429] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.5. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-17 11:47:53,442] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.7.5,  but loading from BentoML version 0.7.5+21.g2b3d2c8.dirty
[2020-05-17 11:47:53,449] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image 

#### Send prediction requeset to the REST API server

```
curl -X POST \
  http://localhost:5000/predict \
  -H 'Content-Type: application/json' \
  -d '[[5.1, 3.5, 1.4, 0.2]]'
```



### Install saved BentoService as PyPI package


In [12]:
!pip install {saved_path}

Processing /Users/bozhaoyu/bentoml/repository/IrisClassifierOnnx/20200517114625_DB7077


  Created wheel for IrisClassifierOnnx: filename=IrisClassifierOnnx-20200517114625_DB7077-py3-none-any.whl size=4367 sha256=44f20f17eeb9944c375f0b2ee634926fcd33210f2b0bfbe3aec0274ac62f8f45
  Stored in directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/pip-ephem-wheel-cache-0c5ca1on/wheels/43/e9/e1/b7902e1c70c2228dab078e6adb3ec1db2afbe1776ebddb7096
Successfully built IrisClassifierOnnx


In [13]:
!IrisClassifierOnnx info

[2020-05-17 11:49:26,394] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.5. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-17 11:49:26,410] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.7.5,  but loading from BentoML version 0.7.5+21.g2b3d2c8.dirty
{
  "name": "IrisClassifierOnnx",
  "version": "20200517114625_DB7077",
  "created_at": "2020-05-17T18:46:41.959251Z",
  "env": {
    "conda_env": "name: bentoml-IrisClassifierOnnx\nchannels:\n- defaults\ndependencies:\n-

In [16]:
!IrisClassifierOnnx run predict --input '[[5.1, 3.5, 1.4, 0.2]]'

[2020-05-17 11:51:32,998] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.7.5. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-17 11:51:33,013] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.7.5,  but loading from BentoML version 0.7.5+21.g2b3d2c8.dirty
[2020-05-17 11:51:33,016] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image 

### Deploy BentoService as REST API server to the cloud


BentoML support deployment to multiply cloud provider services, such as AWS Lambda, AWS Sagemaker, Google Cloudrun and etc. You can find the full list and guide on the documentation site at https://docs.bentoml.org/en/latest/deployment/index.html

For this project, we are going to deploy to AWS lambda

`bentoml lambda deploy` command deploys to AWS lambda with a single command

In [24]:
!bentoml lambda deploy onnx-iris -b IrisClassifierOnnx:{svc.version} --api-name predict --debug

[2020-05-17 12:08:36,339] DEBUG - Creating local YataiService instance
[2020-05-17 12:08:36,544] DEBUG - Upgrading tables to the latest revision
Deploying "IrisClassifierOnnx:20200517114625_DB7077" to AWS Lambda |[2020-05-17 12:08:37,645] DEBUG - Creating s3 bucket: btml-dev-onnx-iris-4d3d6d in region us-west-2
\[2020-05-17 12:08:38,536] DEBUG - Created temporary directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-xszwplmj
[2020-05-17 12:08:38,536] DEBUG - Generating cloudformation template.yaml for lambda project at /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-xszwplmj
[2020-05-17 12:08:38,546] DEBUG - Validating generated template.yaml
[2020-05-17 12:08:38,546] DEBUG - Setting envar "AWS_DEFAULT_REGION" to us-west-2 for subprocess call
-[2020-05-17 12:08:41,120] DEBUG - SAM cmd ['sam', 'validate', '--template-file', '/private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-xszwplmj/template.yaml', '--region', 'us-we

[2020-05-17 12:10:31,457] DEBUG - removing dir: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-xszwplmj/.aws-sam/build/predict/psutil/tests
[2020-05-17 12:10:31,461] DEBUG - removing dir: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-xszwplmj/.aws-sam/build/predict/bentoml/migrations/__pycache__
[2020-05-17 12:10:31,462] DEBUG - removing file: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-xszwplmj/.aws-sam/build/predict/bentoml/migrations/env.pyc
[2020-05-17 12:10:31,462] DEBUG - removing dir: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-xszwplmj/.aws-sam/build/predict/bentoml/migrations/versions/__pycache__
[2020-05-17 12:10:31,463] DEBUG - removing file: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-xszwplmj/.aws-sam/build/predict/bentoml/migrations/versions/a6b00ae45279_add_last_updated_at_for_deployments.pyc
[2020-05-17 12:10:31,464] DEBUG - removing dir: /private

[2020-05-17 12:11:51,919] INFO - ApplyDeployment (onnx-iris, namespace dev) succeeded
Successfully created AWS Lambda deployment onnx-iris
{
  "namespace": "dev",
  "name": "onnx-iris",
  "spec": {
    "bentoName": "IrisClassifierOnnx",
    "bentoVersion": "20200517114625_DB7077",
    "operator": "AWS_LAMBDA",
    "awsLambdaOperatorConfig": {
      "region": "us-west-2",
      "apiName": "predict",
      "memorySize": 1024,
      "timeout": 3
    }
  },
  "state": {
    "state": "RUNNING",
    "infoJson": {
      "endpoints": [
        "https://kfov6vj4kf.execute-api.us-west-2.amazonaws.com/Prod/predict"
      ],
      "s3_bucket": "btml-dev-onnx-iris-4d3d6d"
    },
    "timestamp": "2020-05-17T19:11:52.126490Z"
  },
  "createdAt": "2020-05-17T19:08:36.575258Z",
  "lastUpdatedAt": "2020-05-17T19:08:36.575291Z"
}


In [25]:
!curl -X POST \
  https://kfov6vj4kf.execute-api.us-west-2.amazonaws.com/Prod/predict \
  -H 'Content-Type: application/json' \
  -d '[[5.1, 3.5, 1.4, 0.2]]'

[0]

In [26]:
!bentoml lambda delete onnx-iris

Successfully deleted AWS Lambda deployment "onnx-iris"
